In [4]:

import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

# Carregar os dados
data_train = pd.read_csv("https://media.githubusercontent.com/media/psi3471/datasets/main/disease_prediction/disease_train.csv").drop(columns=["Unnamed: 0"])
data_test = pd.read_csv("https://media.githubusercontent.com/media/psi3471/datasets/main/disease_prediction/disease_test.csv").drop(columns=["Unnamed: 0"])

# Separar os dados de entrada (X) e as saídas (y)
X_train = data_train.iloc[:, :-1].values
y_train = data_train.iloc[:, -1].values
X_test = data_test.iloc[:, :-1].values
y_test = data_test.iloc[:, -1].values


X_train = torch.Tensor(X_train)
y_train = torch.Tensor(y_train)
X_test = torch.Tensor(X_test)
y_test = torch.Tensor(y_test)

# Definir a arquitetura do modelo
class Model(nn.Module):
    def __init__(self, input_size):
        super(Model, self).__init__()
        self.fc1 = nn.Linear(input_size, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 1)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)
        x = self.sigmoid(x)
        return x

# Instanciar o modelo
input_size = X_train.shape[1]
model = Model(input_size)

# Definir a função de perda e o otimizador
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Definir hiperparâmetros e número de épocas
batch_size = 32
num_epochs = 100

# Treinamento do modelo
best_model = None
best_loss = float('inf')

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    # Dividir os dados de treinamento em lotes (batches)
    for i in range(0, len(X_train), batch_size):
        inputs = X_train[i:i+batch_size]
        targets = y_train[i:i+batch_size]

        # Zerar os gradientes do otimizador
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, targets.unsqueeze(1))

        # Backward pass e otimização
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    # Calcular a perda média do epoch
    epoch_loss = running_loss / (len(X_train) / batch_size)

    # Validar o modelo usando os dados de teste
    model.eval()
    with torch.no_grad():
        outputs = model(X_test)
        #
        predicted_labels = (outputs > 0.5).float()

    # Calcular as métricas de desempenho
    accuracy = accuracy_score(y_test, predicted_labels)
    precision = precision_score(y_test, predicted_labels)
    recall = recall_score(y_test, predicted_labels)
    f1 = f1_score(y_test, predicted_labels)

    # Imprimir as métricas de desempenho do modelo
    print(f"Epoch {epoch+1}/{num_epochs} Loss: {epoch_loss:.4f} Accuracy: {accuracy:.4f} Precision: {precision:.4f} Recall: {recall:.4f} F1-score: {f1:.4f}")

    # Armazenar o modelo com menor perda
    if epoch_loss < best_loss:
        best_loss = epoch_loss
        best_model = model.state_dict()

# Carregar o melhor modelo obtido durante o treinamento
model.load_state_dict(best_model)

# Avaliar o modelo final usando os dados de teste
model.eval()
with torch.no_grad():
    outputs = model(X_test)
    predicted_labels = (outputs > 0.5).float()

# Calcular as métricas de desempenho finais
accuracy = accuracy_score(y_test, predicted_labels)
precision = precision_score(y_test, predicted_labels)
recall = recall_score(y_test, predicted_labels)
f1 = f1_score(y_test, predicted_labels)

# Imprimir as métricas de desempenho finais do modelo
print("Final Test Metrics:")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-score: {f1:.4f}")

# Gerar a matriz de confusão
confusion = confusion_matrix(y_test, predicted_labels)
print("Confusion Matrix:")
print(confusion)

Epoch 1/100 Loss: 3.0302 Accuracy: 0.5909 Precision: 0.3654 Recall: 0.3878 F1-score: 0.3762
Epoch 2/100 Loss: 1.1403 Accuracy: 0.5844 Precision: 0.3810 Recall: 0.4898 F1-score: 0.4286
Epoch 3/100 Loss: 1.2189 Accuracy: 0.7273 Precision: 0.6061 Recall: 0.4082 F1-score: 0.4878
Epoch 4/100 Loss: 1.0947 Accuracy: 0.7078 Precision: 0.5500 Recall: 0.4490 F1-score: 0.4944
Epoch 5/100 Loss: 1.0104 Accuracy: 0.6948 Precision: 0.5185 Recall: 0.5714 F1-score: 0.5437
Epoch 6/100 Loss: 1.0027 Accuracy: 0.6948 Precision: 0.5192 Recall: 0.5510 F1-score: 0.5347
Epoch 7/100 Loss: 0.9736 Accuracy: 0.6883 Precision: 0.5102 Recall: 0.5102 F1-score: 0.5102
Epoch 8/100 Loss: 0.9461 Accuracy: 0.7208 Precision: 0.5652 Recall: 0.5306 F1-score: 0.5474
Epoch 9/100 Loss: 0.9283 Accuracy: 0.7273 Precision: 0.5745 Recall: 0.5510 F1-score: 0.5625
Epoch 10/100 Loss: 0.9122 Accuracy: 0.7273 Precision: 0.5745 Recall: 0.5510 F1-score: 0.5625
Epoch 11/100 Loss: 0.8964 Accuracy: 0.7273 Precision: 0.5745 Recall: 0.5510 F1-

Epoch 97/100 Loss: 0.6444 Accuracy: 0.7468 Precision: 0.6250 Recall: 0.5102 F1-score: 0.5618
Epoch 98/100 Loss: 0.6438 Accuracy: 0.7468 Precision: 0.6250 Recall: 0.5102 F1-score: 0.5618
Epoch 99/100 Loss: 0.6431 Accuracy: 0.7468 Precision: 0.6250 Recall: 0.5102 F1-score: 0.5618
Epoch 100/100 Loss: 0.6424 Accuracy: 0.7468 Precision: 0.6250 Recall: 0.5102 F1-score: 0.5618
Final Test Metrics:
Accuracy: 0.7468
Precision: 0.6250
Recall: 0.5102
F1-score: 0.5618
Confusion Matrix:
[[90 15]
 [24 25]]


# Visão geral sobre os resultados
O resultado final inclui a acurácia, precisão, recall, pontuação F1 e matriz de confusão, calculado com base nos dados de teste (data_test) e nos dados de treinamento (data_train). Os formatos dos conjuntos de treinamento e teste são (614, 9) e (154, 9) respectivamente, conforme mencionado.

O conjunto de treinamento possui 614 exemplos e 9 colunas, enquanto o conjunto de teste possui 154 exemplos e 9 colunas. Essas dimensões são consistentes com os resultados que obtive.

O desempenho do modelo foi avaliado utilizando os dados de teste para fazer a inferência e calcular as métricas de avaliação, como a acurácia, precisão, recall e pontuação F1. A matriz de confusão também foi construída com base nos resultados obtidos pela comparação das classificações do modelo com as classificações reais do conjunto de teste.

Aqui está o significado de cada métrica e a interpretação da matriz de confusão:

* Accuracy (Acurácia): É a proporção de exemplos classificados corretamente em relação ao total de exemplos. No no meu caso, o modelo obteve uma acurácia de 0.7468, o que significa que ele classificou corretamente 74.68% dos casos.

* Precision (Precisão): É a proporção de verdadeiros positivos em relação à soma de verdadeiros positivos e falsos positivos. No meu caso, o modelo obteve uma precisão de 0.6250, o que significa que quando ele classificou um exemplo como positivo, em média 62.50% desses casos eram realmente positivos.

* Recall (Revocação): É a proporção de verdadeiros positivos em relação à soma de verdadeiros positivos e falsos negativos. No meu caso, o modelo obteve um recall de 0.5102, o que significa que ele conseguiu identificar corretamente 51.02% dos casos positivos.

* F1-score (Pontuação F1): É a média harmônica entre precisão e recall, fornecendo uma medida balanceada do desempenho do modelo. No meu caso, o F1-score é 0.5618.

A matriz de confusão apresenta as seguintes informações:

* Verdadeiros positivos (True Positives): 90 
* Falsos positivos (False Positives): 15 
* Falsos negativos (False Negatives): 24 
* Verdadeiros negativos (True Negatives): 25

A interpretação da matriz de confusão é a seguinte:

Dos 105 exemplos que são realmente positivos, o modelo corretamente identificou 90 como positivos (verdadeiros positivos) e erroneamente classificou 15 como negativos (falsos negativos). Dos 49 exemplos que são realmente negativos, o modelo corretamente identificou 25 como negativos (verdadeiros negativos) e erroneamente classificou 24 como positivos (falsos positivos). Esses resultados indicam que o modelo possui uma acurácia razoável, mas ainda há espaço para melhorias.